#Lab: Data Management and Databases

#### 1. Import pandas and sqlite3 into this project

In [0]:
import pandas as pd
import sqlite3

#### 2. Mount the google drive folder storing the MIMIC III demo csv files

In [0]:
#mount google drive 
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


#### 3. Create variables that store the following csv files as pandas dataframes (along with any other csvs you might be interested in): 

*   PATIENTS.csv
*   ADMISSIONS.csv
*   PRESCRIPTIONS.csv
*   SERVICES.csv
*   D_ICD_DIAGNOSES.csv
*   DIAGNOSES_ICD.csv

In [0]:
patients = pd.read_csv("/content/gdrive/My Drive/[YCMI_CBDS Summer Course] Data/mimic_iii/PATIENTS.csv")
admissions = pd.read_csv("/content/gdrive/My Drive/[YCMI_CBDS Summer Course] Data/mimic_iii/ADMISSIONS.csv")
prescriptions = pd.read_csv("/content/gdrive/My Drive/[YCMI_CBDS Summer Course] Data/mimic_iii/PRESCRIPTIONS.csv")
services = pd.read_csv("/content/gdrive/My Drive/[YCMI_CBDS Summer Course] Data/mimic_iii/SERVICES.csv")
diagnosis_icds = pd.read_csv("/content/gdrive/My Drive/[YCMI_CBDS Summer Course] Data/mimic_iii/D_ICD_DIAGNOSES.csv")
diagnosis_events = pd.read_csv("/content/gdrive/My Drive/[YCMI_CBDS Summer Course] Data/mimic_iii/DIAGNOSES_ICD.csv")

#### 4. Add to the existing variable assignments below to create code that will establish a database connection (name the database "lab.db"), and create a database cursor.

In [0]:
conn = sqlite3.connect('lab.db')
cursor = conn.cursor()

#### 5. Add the csvs stored as variables in prompt 3 to lab.db using the .to_sql.() method

In [0]:
patients.to_sql("patients", conn, if_exists="replace", index = False)
admissions.to_sql("admissions", conn, if_exists="replace", index = False)
prescriptions.to_sql("prescriptions", conn, if_exists="replace", index = False)
services.to_sql("services", conn, if_exists="replace", index = False)
diagnosis_icds.to_sql("diagnosis_icds", conn, if_exists="replace", index = False)
diagnosis_events.to_sql("diagnosis_events", conn, if_exists="replace", index = False)

#### 6. Write an sql query that pulls the icd9_code and short_title columns from the table created from the d_icd_diagnoses csv.

In [0]:
pd.read_sql_query('SELECT icd9_code, short_title from diagnosis_icds', conn)

,icd9_code,short_title
0,01716,Erythem nod tb-oth test
1,01720,TB periph lymph-unspec
2,01721,TB periph lymph-no exam
3,01722,TB periph lymph-exam unk
4,01723,TB periph lymph-micro dx
...,...,...
14562,V8712,Contact/exposure benzene
14563,V8719,Cont/exp haz aromat NEC
14564,V872,Cont/exp hazard chem NEC
14565,V8731,Contact/exposure mold


#### 7. Write a query that results in the number of subjects per each generic drug name, and order by route (descending). 

In [0]:
pd.read_sql_query('''SELECT route, drug_name_generic, COUNT(subject_id) FROM prescriptions 
WHERE dose_unit_rx = "mg"
AND dose_val_rx >= 400
GROUP BY drug_name_generic
ORDER BY route DESC
;''', conn)

,route,drug_name_generic,COUNT(subject_id)
0,SC,Enoxaparin Sodium,15
1,SC,Sumatriptan Succinate,2
2,PR,Acetaminophen (Rectal),16
3,PO/NG,Acetaminophen,43
4,PO/NG,Acetaminophen (Liquid),18
...,...,...,...
148,IV,Morphine Sulfate (Syringe),3
149,IV,Ondansetron,9
150,IV,Prochlorperazine,2
151,IV,Vancomycin,1


#### 8. Surgury-related event (hint: surgeries are noted in the services table), find all of the diagonse names (short titles) tied to these events. Display only emergency admission types.

In [0]:
pd.read_sql_query('''SELECT curr_service, services.subject_id, short_title
FROM services
LEFT JOIN admissions on services.subject_id = admissions.subject_id
LEFT JOIN diagnosis_events on services.subject_id = diagnosis_events.subject_id
LEFT JOIN diagnosis_icds on diagnosis_events.icd9_code = diagnosis_icds.icd9_code
WHERE services.curr_service LIKE "%SURG%" AND admission_type = "EMERGENCY";''', conn)

,curr_service,subject_id,short_title
0,SURG,10033,Tox multnod goit no cris
1,SURG,10033,DMII wo cmp nt st uncntr
2,SURG,10033,Pure hypercholesterolem
3,SURG,10033,Hyperpotassemia
4,SURG,10033,Depressive disorder NEC
...,...,...,...
4533,SURG,44228,Anemia NOS
4534,SURG,44228,Phlbts sprfc vn up extrm
4535,SURG,44228,"Pneumonia, organism NOS"
4536,SURG,44228,Septic shock


9. Write a query that will return diagnoses that appear more than 15 times. 

In [0]:
pd.read_sql('''SELECT icd9_code, COUNT(icd9_code)
FROM diagnosis_events
GROUP BY icd9_code
HAVING COUNT(icd9_code) > 15''', conn)

,icd9_code,COUNT(icd9_code)
0,0389,24
1,2449,23
2,25000,31
3,2724,29
4,2760,17
5,2761,17
6,2762,16
7,2859,25
8,311,16
9,4019,53


#### 10. Write a query that compares ethnicity and insurance type in some way. Compare your inqueries and results with others in your working group.

In [0]:
#This example compares the distribution of insurance types across represented 
#ethnicities in the database
pd.read_sql('''SELECT ethnicity, insurance, COUNT(insurance)
FROM admissions
GROUP BY ethnicity, insurance;''', conn)

,ethnicity,insurance,COUNT(insurance)
0,AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGN...,Medicaid,2
1,ASIAN,Medicare,2
2,BLACK/AFRICAN AMERICAN,Medicare,7
3,HISPANIC OR LATINO,Medicare,2
4,HISPANIC/LATINO - PUERTO RICAN,Medicare,15
5,OTHER,Medicare,1
6,OTHER,Private,2
7,UNABLE TO OBTAIN,Medicare,1
8,UNKNOWN/NOT SPECIFIED,Medicaid,2
9,UNKNOWN/NOT SPECIFIED,Medicare,4
